In [5]:
from src.data import make_dataset
from transformers import pipeline
from sklearn.metrics import classification_report
import torch
import pandas as pd
torch.cuda.is_available()

True

In [6]:
# input_file = "../../data/raw/reviews.csv"
# train_output_file = "../../data/processed/train_final_processed_reviews.csv"
# test_output_file = "../../data/processed/test_final_processed_reviews.csv"
# X_train, X_test, y_train, y_test = make_dataset.main(input_file, train_output_file, test_output_file)
train = pd.read_csv("../../data/processed/train_final_processed_reviews.csv")
test = pd.read_csv("../../data/processed/test_final_processed_reviews.csv")
train.head()

,Unnamed: 0,time,text,cleaned_text,lowercase_count,uppercase_count,uppercase_words,uppercase_ratio,punc_count,num_digits,num_verbs,num_nouns,num_tokens_cleaned,num_tokens_raw,num_words_misspelled,polarity,subjectivity,num_pos_words,num_neg_words,sentiment
0,4979,11/12/19,Disappointed. The big boxes had a very differ...,disappointed big box different flavor small bo...,23,0,NaN,0.000000,3,0,0,3,9,24,0,0.2732,0.556481,0,0,negative
1,659,6/3/21,This is a wonderfully refreshing citrus drink....,wonderfully refresh citrus drink sweet sour en...,87,0,NaN,0.000000,7,0,8,13,40,88,2,0.9747,0.653889,2,0,positive
2,1097,22/3/18,I just brewed my first cup and the aroma and f...,brew first cup aroma flavor speak autumn harve...,27,1,LOVE,0.032258,3,0,2,7,15,31,0,0.7902,0.491111,1,0,positive
3,5351,30/7/21,"It taste okay, but they are not soft and the ...",taste okay soft taste stay long would buy,24,0,NaN,0.000000,3,0,2,3,8,25,2,0.2263,0.416667,0,0,negative
4,2209,20/4/21,These are good chips. I like the texture and ...,good chip like texture easy chew regular torti...,26,0,NaN,0.000000,3,0,0,6,12,27,0,0.8883,0.627564,0,0,positive


In [10]:
X_train = train.drop('sentiment', axis=1)
X_test = test.drop('sentiment', axis=1)
y_train = train.sentiment
y_test = test.sentiment

In [11]:
X_test.head()

,Unnamed: 0,time,text,cleaned_text,lowercase_count,uppercase_count,uppercase_words,uppercase_ratio,punc_count,num_digits,num_verbs,num_nouns,num_tokens_cleaned,num_tokens_raw,num_words_misspelled,polarity,subjectivity,num_pos_words,num_neg_words
0,940,8/6/21,"This tea is amazing, I cannot believe the pric...",tea amazing believe price price pay less half ...,38,1,FAST,0.02381,6,0,0,9,17,42,0,0.9468,0.583333,4,0
1,3920,4/9/21,I've tried a handful of other brands of French...,try handful brand french roast find one tully ...,41,0,NaN,0.00000,4,2,2,6,17,43,2,0.6249,0.587500,1,0
2,5038,9/12/20,I would recommend not buying these nuts if you...,would recommend buying nut look really good ta...,70,0,NaN,0.00000,7,0,5,13,32,72,0,0.8399,0.563719,0,0
3,3432,21/4/21,"I wish these came in larger quantities, if I c...",wish come large quantity could cuppa everyday ...,20,0,NaN,0.00000,3,0,3,2,10,23,0,0.4019,0.514286,0,0
4,2376,13/1/20,Has a smooth taste like it claims without any ...,smooth taste like claim without unpleasant met...,68,0,NaN,0.00000,5,0,2,13,36,71,3,0.9408,0.605000,0,0


In [12]:
y_test.head()

0    positive
1    positive
2    negative
3    positive
4    positive
Name: sentiment, dtype: object

In [54]:
%%time
sentiment_pipeline = pipeline(model = "distilbert-base-uncased-finetuned-sst-2-english")
data = X_test.cleaned_text.to_list()
results = sentiment_pipeline(data)

CPU times: total: 6.41 s
Wall time: 6.37 s


In [55]:
results[0:5]

[{'label': 'POSITIVE', 'score': 0.9995902180671692},
 {'label': 'NEGATIVE', 'score': 0.9640170335769653},
 {'label': 'POSITIVE', 'score': 0.985977292060852},
 {'label': 'POSITIVE', 'score': 0.673072874546051},
 {'label': 'POSITIVE', 'score': 0.745204746723175}]

In [30]:
labels = [result['label'] for result in results]

In [51]:
y_pred = [1 if label=='POSITIVE' else 0 for label in labels]
y_true = [1 if label=='positive' else 0 for label in y_test]
print(y_pred[0:5])
print(y_true[0:5])
print(classification_report(y_true, y_pred))

[1, 0, 1, 1, 1]
[1, 1, 0, 1, 1]
              precision    recall  f1-score   support

           0       0.47      0.87      0.61       283
           1       0.93      0.65      0.77       806

    accuracy                           0.71      1089
   macro avg       0.70      0.76      0.69      1089
weighted avg       0.81      0.71      0.73      1089



In [13]:
data = X_test.text.to_list()
data = [x[:512] if len(x)>512 else x for x in data]

In [23]:
%%time
specific_model = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment", device=0)
# specific_model = pipeline(model="Seethal/sentiment_analysis_generic_dataset", device=0)
# specific_model = pipeline(model="siebert/sentiment-roberta-large-english", device=0)
results = specific_model(data)

CPU times: total: 10.1 s
Wall time: 10.8 s


In [24]:
results[0:5]

[{'label': '5 stars', 'score': 0.9336758852005005},
 {'label': '5 stars', 'score': 0.4628945291042328},
 {'label': '2 stars', 'score': 0.555162250995636},
 {'label': '4 stars', 'score': 0.5062201619148254},
 {'label': '4 stars', 'score': 0.6108953952789307}]

In [32]:
labels = [result['label'] for result in results]
y_pred = [1 if label in ['5 stars', '4 stars', '3 stars'] else 0 for label in labels]
# y_pred = [1 if label=='POSITIVE' else 0 for label in labels]
# y_pred = [1 if label=='LABEL_2' else 0 for label in labels]
# y_pred = [0 if label=='LABEL_0' else 1 for label in labels]
y_true = [1 if label=='positive' else 0 for label in y_test]
print(y_pred[0:10])
print(y_true[0:10])
print(classification_report(y_true, y_pred))

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       283
           1       0.94      0.94      0.94       806

    accuracy                           0.92      1089
   macro avg       0.89      0.89      0.89      1089
weighted avg       0.92      0.92      0.92      1089

